In [29]:
import timm
import torch
import onnx
import datetime

In [30]:
! python -c "from timm import list_models; print(list_models(pretrained=True)[:5])"

['bat_resnext26ts.ch_in1k', 'beit_base_patch16_224.in22k_ft_in22k', 'beit_base_patch16_224.in22k_ft_in22k_in1k', 'beit_base_patch16_384.in22k_ft_in22k_in1k', 'beit_large_patch16_224.in22k_ft_in22k']


In [31]:
# 一共有1298个模型
model_pretrain_list = timm.list_models(pretrained=True)
print(len(model_pretrain_list))


1298


In [32]:
# 查看所有的swin-transformer模型
all_densnet_models = timm.list_models("*swin*")
all_densnet_models


['swin_base_patch4_window7_224',
 'swin_base_patch4_window12_384',
 'swin_large_patch4_window7_224',
 'swin_large_patch4_window12_384',
 'swin_s3_base_224',
 'swin_s3_small_224',
 'swin_s3_tiny_224',
 'swin_small_patch4_window7_224',
 'swin_tiny_patch4_window7_224',
 'swinv2_base_window8_256',
 'swinv2_base_window12_192',
 'swinv2_base_window12to16_192to256',
 'swinv2_base_window12to24_192to384',
 'swinv2_base_window16_256',
 'swinv2_cr_base_224',
 'swinv2_cr_base_384',
 'swinv2_cr_base_ns_224',
 'swinv2_cr_giant_224',
 'swinv2_cr_giant_384',
 'swinv2_cr_huge_224',
 'swinv2_cr_huge_384',
 'swinv2_cr_large_224',
 'swinv2_cr_large_384',
 'swinv2_cr_small_224',
 'swinv2_cr_small_384',
 'swinv2_cr_small_ns_224',
 'swinv2_cr_small_ns_256',
 'swinv2_cr_tiny_224',
 'swinv2_cr_tiny_384',
 'swinv2_cr_tiny_ns_224',
 'swinv2_large_window12_192',
 'swinv2_large_window12to16_192to256',
 'swinv2_large_window12to24_192to384',
 'swinv2_small_window8_256',
 'swinv2_small_window16_256',
 'swinv2_tiny_wi

### 加载swin模型，并进行参数的配置

In [33]:
model = timm.create_model('swin_tiny_patch4_window7_224', num_classes=1000, pretrained=True, pretrained_cfg_overlay=dict(file='/mnt/share_disk/cdd/pretrained_models/swin_tiny_patch4_window7_224.pth'))
model.default_cfg

{'url': 'https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth',
 'file': '/mnt/share_disk/cdd/pretrained_models/swin_tiny_patch4_window7_224.pth',
 'hf_hub_id': 'timm/swin_tiny_patch4_window7_224.ms_in1k',
 'architecture': 'swin_tiny_patch4_window7_224',
 'tag': 'ms_in1k',
 'custom_load': False,
 'input_size': (3, 224, 224),
 'fixed_input_size': True,
 'interpolation': 'bicubic',
 'crop_pct': 0.9,
 'crop_mode': 'center',
 'mean': (0.485, 0.456, 0.406),
 'std': (0.229, 0.224, 0.225),
 'num_classes': 1000,
 'pool_size': (7, 7),
 'first_conv': 'patch_embed.proj',
 'classifier': 'head.fc',
 'license': 'mit'}

In [25]:
dummy_input = torch.randn(1, 3, 224, 224)
model.eval()
now = datetime.datetime.now()
model_file_name = f"/mnt/share_disk/cdd/export_onnx_models/swin_tiny_patch4_window7_224_{now.strftime('%Y%m%d_%H%M%S')}.onnx"

torch.onnx.export(model, dummy_input, model_file_name, opset_version=11)

/opt/conda/envs/timm/lib/python3.8/site-packages/torch/__init__.py:1493: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert condition, message


In [20]:
def onnx_simplify(path):
    import onnx
    from onnxsim import simplify
    
    # simplify the onnx model & load onnx model
    onnx_model = onnx.load(path)  
    model_simp, check = simplify(onnx_model)
    assert check, "Simplified ONNX model could not be validated"
    
    output_path = path.split(".")[0] + "_onnxsim." + path.split(".")[-1]
    
    onnx.save(model_simp, output_path)
    
    print("The simplified ONNX model is saved to {}".format(output_path))

In [34]:
# onnxsim 简化会出问题，error
onnx_simplify("/mnt/share_disk/cdd/pytorch-image-models/swin-transformer_new.onnx")

InferenceError: [ShapeInferenceError] (op_type:Transpose, node name: /layers/layers.0/blocks/blocks.0/Transpose_1): [ShapeInferenceError] Inferred shape and existing shape differ in rank: (6) vs (0)